# VQE Exercise in PennyLane

To show the different steps to follow in this algorithm and how implement it in `PennyLane` we are going to minimize the following function: 

$$ f(x_{0}, x_{1}, x_{2}) = 2x_{0} - 4x_{0}x_{1} + 3x_{1}x_{2}$$

The first we have to do is transform it into the Ising notation, which means pass our variables from binary notation to another which 0 = 1 and 1 = -1, and the way to do that is with the change: 

$$x = \frac{1-z}{2} $$

So, the function above in the Ising notation is: 

$$ g(z_{0}, z_{1}, z_{2}) = \frac{3}{4} + \frac{1}{4}z_{1} - \frac{3}{4}z_{2} - z_{0}z_{1} + \frac{3}{4}z_{1}z_{2}$$

By teasting, we know in advance the solution is:

<ul>
    <li> 1 1 0 (QUBO) </li>
    <li> -1 -1 1 (ISING) </li>
</ul>

So lets proceed with programming

In [ ]:
import pennylane as qml
import numpy as np

In [ ]:
# Define the number of qubits we are going to use
variables = 3

# Select the device where we are going to execute the circuit
dev = qml.device('default.qubit', wires = variables, shots = 1000)

# Create the state wavefunction
@qml.template
def phi(w):
    qml.RX(w[0], wires = 0)
    qml.RX(w[1], wires = 1)
    qml.RX(w[2], wires = 2)
    
# Create differents subcircuits associated to our hamiltonian:

# Circuit associated to Identity
@qml.qnode(dev)
def circ1(w):
    phi(w)
    return qml.exval(qml.Identity(wires = 0)
                     @qml.Identity(wires = 1)
                     @qml.Identity(wires = 2))
    
# Circuit associated to z1 
@qml.qnode(dev)
def circ2(w):
    phi(w)
    return qml.exval(qml.Identity(wires = 0)
                     @qml.PauliZ(wires = 1)
                     @qml.Identity(wires = 2))
    #return qml.PauliZ(wires = 1)
    
# Circuit associated to z2 
@qml.qnode(dev)
def circ3(w):
    phi(w)
    return qml.exval(qml.Identity(wires = 0)
                     @qml.Identity(wires = 1)
                     @qml.PauliZ(wires = 2))
    #return qml.PauliZ(wires = 2)
    
# Circuit associated to z0z1 
@qml.qnode(dev)
def circ4(w):
    phi(w)
    return qml.exval(qml.PauliZ(wires = 0)
                     @qml.PauliZ(wires = 1)
                     @qml.Identity(wires = 2))
    #return qml.PauliZ(wires = 0) @qml.PauliZ(wires = 1)

# Circuit associated to z1z2
@qml.qnode(dev)
def circ5(w):
    phi(w)
    return qml.exval(qml.Identity(wires = 0)
                     @qml.PauliZ(wires = 1)
                     @qml.PauliZ(wires = 2))
    #return qml.PauliZ(wires = 1) @qml.PauliZ(wires = 2)
 
# Assamble all elements of the circuit.    
def value(w): 
    return 3*circ1(w)/4 + circ2(w)/4 - 3*circ3(w)/4 - circ4(w) + 3*circ5(w)/4

# Initialize w randomly
w = qml.numpy.random.rand(3) * 2 * np.pi

# Define a gradient
gradient_fn_w = qml.grad(value, argnum = 0)

lr = 0.005
for epoch in range(501):
    w = w - lr*gradient_fn_w(w)
    if epoch % 50 == 0:
        print('epoch', epoch, 'loss', value(w))
        
@qml.qnode(qml.device('default.qubit', wires = varaibles, shots = 1))

def solucion(w):
    phi(w)
    return [qml.sample(qml.PauliZ(i)) for i in range(3)]

solucion(w)